In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['fruits-360_dataset']


In [38]:
import numpy as np
import cv2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import glob
import os
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cluster import KMeans

In [39]:
# load data
fruit_images = []
labels = [] 
for fruit_dir_path in glob.glob("../input/*/fruits-360/Training/*"):
    fruit_label = fruit_dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        image = cv2.resize(image, (45, 45))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        fruit_images.append(image)
        labels.append(fruit_label)
fruit_images = np.array(fruit_images)
labels = np.array(labels)

In [40]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
label_to_id_dict

{'Apple Braeburn': 0,
 'Apple Crimson Snow': 1,
 'Apple Golden 1': 2,
 'Apple Golden 2': 3,
 'Apple Golden 3': 4,
 'Apple Granny Smith': 5,
 'Apple Pink Lady': 6,
 'Apple Red 1': 7,
 'Apple Red 2': 8,
 'Apple Red 3': 9,
 'Apple Red Delicious': 10,
 'Apple Red Yellow 1': 11,
 'Apple Red Yellow 2': 12,
 'Apricot': 13,
 'Avocado': 14,
 'Avocado ripe': 15,
 'Banana': 16,
 'Banana Lady Finger': 17,
 'Banana Red': 18,
 'Cactus fruit': 19,
 'Cantaloupe 1': 20,
 'Cantaloupe 2': 21,
 'Carambula': 22,
 'Cherry 1': 23,
 'Cherry 2': 24,
 'Cherry Rainier': 25,
 'Cherry Wax Black': 26,
 'Cherry Wax Red': 27,
 'Cherry Wax Yellow': 28,
 'Chestnut': 29,
 'Clementine': 30,
 'Cocos': 31,
 'Dates': 32,
 'Granadilla': 33,
 'Grape Blue': 34,
 'Grape Pink': 35,
 'Grape White': 36,
 'Grape White 2': 37,
 'Grape White 3': 38,
 'Grape White 4': 39,
 'Grapefruit Pink': 40,
 'Grapefruit White': 41,
 'Guava': 42,
 'Hazelnut': 43,
 'Huckleberry': 44,
 'Kaki': 45,
 'Kiwi': 46,
 'Kohlrabi': 47,
 'Kumquats': 48,
 'Lem

In [41]:
id_to_label_dict = {v:k for k,v in label_to_id_dict.items()}
id_to_label_dict

{0: 'Apple Braeburn',
 1: 'Apple Crimson Snow',
 2: 'Apple Golden 1',
 3: 'Apple Golden 2',
 4: 'Apple Golden 3',
 5: 'Apple Granny Smith',
 6: 'Apple Pink Lady',
 7: 'Apple Red 1',
 8: 'Apple Red 2',
 9: 'Apple Red 3',
 10: 'Apple Red Delicious',
 11: 'Apple Red Yellow 1',
 12: 'Apple Red Yellow 2',
 13: 'Apricot',
 14: 'Avocado',
 15: 'Avocado ripe',
 16: 'Banana',
 17: 'Banana Lady Finger',
 18: 'Banana Red',
 19: 'Cactus fruit',
 20: 'Cantaloupe 1',
 21: 'Cantaloupe 2',
 22: 'Carambula',
 23: 'Cherry 1',
 24: 'Cherry 2',
 25: 'Cherry Rainier',
 26: 'Cherry Wax Black',
 27: 'Cherry Wax Red',
 28: 'Cherry Wax Yellow',
 29: 'Chestnut',
 30: 'Clementine',
 31: 'Cocos',
 32: 'Dates',
 33: 'Granadilla',
 34: 'Grape Blue',
 35: 'Grape Pink',
 36: 'Grape White',
 37: 'Grape White 2',
 38: 'Grape White 3',
 39: 'Grape White 4',
 40: 'Grapefruit Pink',
 41: 'Grapefruit White',
 42: 'Guava',
 43: 'Hazelnut',
 44: 'Huckleberry',
 45: 'Kaki',
 46: 'Kiwi',
 47: 'Kohlrabi',
 48: 'Kumquats',
 49: 

In [42]:
# transform the all the labels to ids
labels_id = np.array([label_to_id_dict[x] for x in labels])
labels_id[:10]

array([59, 59, 59, 59, 59, 59, 59, 59, 59, 59])

In [43]:
# fruit_images is X and labels_id is y
# reduce the feature dimentions by PCA
scaler = StandardScaler()
image_scaled = scaler.fit_transform([i.flatten() for i in fruit_images]) 

In [45]:
pca = PCA(n_components=50)
image_PCA = pca.fit_transform(image_scaled)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(image_PCA, labels_id, test_size=0.25, random_state=42)

In [47]:
forest = RandomForestClassifier(n_estimators=10)
forest = forest.fit(X_train, y_train)
y_predict = forest.predict(X_test)

In [48]:
precision = accuracy_score(y_predict, y_test) * 100
print("Accuracy with RandomForest: {0:.6f}".format(precision))

Accuracy with RandomForest: 99.676570


In [50]:
svm_clf = svm.SVC()
svm_clf = svm_clf.fit(X_train,y_train)
y_predict = svm_clf.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [51]:
precision = accuracy_score(y_predict, y_test) * 100
print("Accuracy with SVM: {0:.6f}".format(precision))

Accuracy with SVM: 93.892441


In [ ]:
from sklearn.model_selection import cross_val_score
scores_forest = cross_val_score(forest, image_PCA, labels_id, cv=5)

In [53]:
scores_forest

NameError: name 'scores_forest' is not defined